<img width="200" style="float:left" 
     src="https://upload.wikimedia.org/wikipedia/commons/f/f3/Apache_Spark_logo.svg" /> 

<img width="500" style="float:center" 
     src="https://images6.alphacoders.com/119/1191374.jpg" />

# GraphFrames
* [Business Case](#0)
* [1. Setup](#1) 
  * [1.1 Start Hadoop](#1.1)
  * [1.2 Spark Installation](#1.2)
  * [1.3 Create SparkSession](#1.3)
* [2. Twitter & SquidGame](#2) 
  * [2.1 Import the data into a Spark DataFrame ](#2.1)
  * [2.2 Data processing](#2.2)
  * [2.3 Create the nodes & edges DataFrame](#2.3)
  * [2.4 Create the GraphFrame](#2.4)
* [3. Analytics](#3)
  * [3.1. Conversation: Which are the top 10 most common account pairs in the SquidGame conversation?](#3.1)
  * [3.2. Conversation: Which are the top 10 accounts with more triangles in the SquidGame conversation?](#3.2)
  * [3.3. Conversation: How many communities/clusters are in the SquidGame conversation?](#3.3)
  * [3.4. Conversation: Which are the top 5 most important accounts in the SquidGame conversation?](#3.4)
  * [3.5. Conversation: Which are the top 5 accounts that mentioned other accounts the most in the SquidGame conversation?](#3.5)
  * [3.6. Conversation: Which are the top 5 accounts that were mentioned the most in the SquidGame conversation?](#3.6)
* [4. Export graph data](#4)
* [5. Finalize the exercise](#5)
  * [5.1 Stop the Hadoop service](#5.1)

<a id='0'></a>
## Business Case

<p>To further understand user's behavior and find valuable insights for the future marketing campaign of the second season of Squid Game, Netflix has asked Team H, as analytics consultants, to set up advanced analysis of a batch dataset containing 80K tweets to provide insights on the relationship between accounts and the mentions associated to those accounts. Hence, to find valuable information regarding the batch dataset, Team H has established 6 key business questions aligned with the Marketing and Sales strategy following two different pillars: who are the users? Which is their relationship?. Therefore, this Jupyter Notebook is a compilation of ready-to-query consumer-insights questions that aims to support those strategic teams with insights into the Squid Game conversations.

</div>

<ul class="roman">
     <li> Quick insights on the general conversation relationships </li>
     <ul class="square">
 <li>1. Conversation: Which are the top 10 most common account pairs in the SquidGame conversation?</li>
 <li>2. Conversation: Which are the top 10 accounts with more triangles in the SquidGame conversation?</li>
 <li>3. Conversation: How many communities/clusters are in the SquidGame conversation?
 </ul>
 <li> Quick insights on the individual conversation:
 <ul class="square">
 <li>4. Conversation Background: Which are the top 5 most important accounts in the SquidGame conversation?</li>
 <li>5. Conversation Background: Which are the top 5 accounts that mentioned other accounts the most in the SquidGame conversation?</li>
 <li>6. Conversation Background: Which are the top 5 accounts that were mentioned the most in the SquidGame conversation?</li>

<a id='1'></a>
## 1. Setup

Since we are going to process data stored from HDFS let's start the service

<a id='1.1'></a>
### 1.1 Start Hadoop 
<p>Dear Marketing Team, before starting to use this notebook please be sure that you log into the course environment by:</p>
<ul>
    <li><p><b>Start the Hadoop service</b>:
        <br/><em>\$ hadoop-start.sh</p></em><br/></li>

<a id='1.1'></a>
### 1.1 Spark Installation
<p>This step is required for working with the data.

In [1]:
import findspark
findspark.init()

<p> We change pandas DataFrame max column width and row to improve data displaying.

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

<a id='1.2'></a>
### 1.2 Create the SparkSession
<p>By setting this environment variable we can include extra libraries in our Spark cluster. We will add GraphFrames since this library is not in spark core.

In [3]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages "graphframes:graphframes:0.8.2-spark3.2-s_2.12" --jars /opt/hive3/lib/hive-hcatalog-core-3.1.2.jar pyspark-shell'

In [4]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
import random

<a id='1.3'></a>
### 1.3 SparkSession
<p>By setting this environment variable we can include extra libraries in our Spark cluster.<br/>

In [5]:
spark = (SparkSession.builder
    .appName("Squid Game - Group G")
    .config("spark.sql.warehouse.dir","hdfs://localhost:9000/datalake")
    .config("spark.sql.legacy.timeParserPolicy","LEGACY")
    .enableHiveSupport()
    .getOrCreate())

Ivy Default Cache set to: /home/osbdet/.ivy2/cache
The jars for the packages stored in: /home/osbdet/.ivy2/jars
:: loading settings :: url = jar:file:/opt/spark3/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-10b22930-6f14-45eb-b1fb-4a55ed78f3a0;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 409ms :: artifacts dl 8ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	------------------------------------------------------------

In [6]:
from pyspark.sql.types import *
batch_customschema = StructType([
StructField(
 "name", StringType(), True),StructField(
 "location", StringType(), True),StructField(
 "description", StringType(), True),StructField(
 "twitter_created_at", StringType(), True),StructField(
 "followers_count", LongType(), True),StructField(
 "friends_count", LongType(), True),StructField(
 "favourites_count", LongType(), True),StructField(
 "verified", BooleanType(), True),StructField(
 "created_at", StringType(), True),StructField(
 "text", StringType(), True),StructField(
 "source", StringType(), True),StructField(
 "retweeted", BooleanType(), True)])

<a id='2'></a>
## 2. Twitter & SquidGame
Here we will extract and clean the necessary pieces of data within our dataset and which is going to be used in the 'Analytics" step

<a id='2.1'></a>
### 2.1 Import the data into a Spark DataFrame

In [7]:
# We import the Squid Game conversation from twitter into a spark DataFrame ('squidgameDF').
squidgameDF = spark.read.format("csv").options(header="true", multiline="true")\
    .schema(batch_customschema).load("hdfs://localhost:9000/datalake/raw")


<a id='2.2'></a>
### 2.2 Exploratory data analysis 

#### 1. Who are the users? Verified distinct accounts that talked about SquidGame with highest following:

In [8]:
most_followed_df = (squidgameDF
          .groupBy("name")
          .agg(max("followers_count").alias("followers_count"))
          .orderBy(desc("followers_count"))
          .limit(5))
most_followed_df.toPandas()

# They ended up being the same as the verified! 

,name,followers_count
0,9GAG,16846417
1,Vogue Magazine,14359408
2,billboard,12588047
3,E! News,11710347
4,Twitter TV,9207638


#### 2. Who are the users? Ratio of Verified accounts in the conversation of SquidGame:

In [9]:
verified_ratio_df = squidgameDF.groupBy("verified").agg(count("*").alias("Count")).sort(col("Count").desc())

verified_ratio_df.toPandas()

# 4% of the users tweeting about SquidGame have verified accounts
# 95% have unverified accounts

,verified,Count
0,False,76411
1,True,3292
2,None,1981


#### 3. Who are the users? Popularity of Twitter users on the Top 10 locations:

In [10]:
import pyspark.sql.functions as f

In [11]:
size_summary = squidgameDF.groupBy("verified")\
        .agg(
            f.count("*").alias("Count"),
            f.max("followers_count").alias("Max_Followers_Count"),
            f.round(f.avg("followers_count")).alias("Avg_followers_count"),
            f.min("followers_count").alias("Min_Followers_Count"))\
        .sort(f.col("Count").desc())\
        .show(10)

size_summary

# Here we can see that one of the verified accounts actually has only 14 followers, showing that being verified does not mean large following.

+--------+-----+-------------------+-------------------+-------------------+
|verified|Count|Max_Followers_Count|Avg_followers_count|Min_Followers_Count|
+--------+-----+-------------------+-------------------+-------------------+
|   false|76411|             944282|             2226.0|                  0|
|    true| 3292|           16846417|           383938.0|                 14|
|    null| 1981|              52912|             8494.0|                  1|
+--------+-----+-------------------+-------------------+-------------------+



#### 4. Who are the users? Top 5 common users with more tweets published about SquidGame:


In [12]:
top_users_df = (squidgameDF.groupBy("name")
          .agg(count("*").alias("Count"))
          .orderBy(desc("Count"))
          .limit(5))
top_users_df.toPandas()

,name,Count
0,Sale X Low,395
1,John Doge,263
2,7-Ainu,170
3,Ofofonobs,140
4,havingalaff,110


<a id='2.3'></a>
### 2.3. Data processing

In [13]:
# We create a pandas DataFrame ('squidDF') for processing.
squidDF = squidgameDF.toPandas()

In [14]:
# We obtain an insight on how many tweets we are going to process in our analysys.
squidDF['text'].count()

80905

In [15]:
# We extract the mentions(@) and we add them to a new column named 'mentions'.
squidDF['mentions'] = squidDF['text'].str.findall(r'[＠@]([^][\s#<>|{}]+)')

# We use the function .explode() to unstack the obtained list of mentions into a row containing only one mention. 
squidDF = squidDF.explode('mentions')

# We drop all the tweets with no mentions in our pandas DataFrame. 
squidDF = squidDF.dropna()

In [16]:
# We obtain a high-level oeverview of the information we have.
squidDF.head(5)

,name,location,description,twitter_created_at,followers_count,friends_count,favourites_count,verified,created_at,text,source,retweeted,mentions
5,Peyman 🅚🅐🅘,United Kingdom,Official @KardiaChain $KAI Ambassador\nMarketing Advisor @kephigallery\nGraphics and Film Artist https://t.co/OsTOSEKrHt,2018-01-27 12:07:31+00:00,546.0,318.0,6265.0,False,2021-10-06 12:04:54+00:00,$THG\nGoing to explode to 4B Marketcap very soon.\nThe world first MOBA\nThis game is on another level!\n@KardiaChain… https://t.co/4Y2bO088JM,Twitter for Android,False,KardiaChain…
14,Yemzxy,"Lagos, Nigeria",Am special because am a triplet,2021-06-11 16:47:29+00:00,39.0,160.0,12.0,False,2021-10-06 12:02:57+00:00,The saga continues \nGet data cheap @honourworldng \nMessage us on https://t.co/0ehyV9EP4w\n#twitch #SquidGame #2baba… https://t.co/ASDjPpp2EH,Twitter for Android,False,honourworldng
26,EarlyGame,Germany,Own Your Game,2019-11-13 14:56:55+00:00,49381.0,153.0,963.0,False,2021-10-06 12:01:25+00:00,Season 2? 😂 @Xbox #SquidGame https://t.co/OoAAie9EaW,PromoRepublic,False,Xbox
31,Charlie,United Kingdom,Aspiring Youtuber/Content creator.\nDisclaimer: Charlogical is a Brand\nThe content posted is reviewed and published by the Charlogical Team\nTeam@charlogical.com,2020-02-07 22:43:29+00:00,24.0,47.0,463.0,False,2021-10-06 12:00:26+00:00,I Didn't mean to push her off - Squid Game Roblox https://t.co/J6olOsZMj5 @youtube @YouTubeGaming @roblox\n\n#roblox… https://t.co/oE9P1UU2iE,Twitter Web App,False,youtube
31,Charlie,United Kingdom,Aspiring Youtuber/Content creator.\nDisclaimer: Charlogical is a Brand\nThe content posted is reviewed and published by the Charlogical Team\nTeam@charlogical.com,2020-02-07 22:43:29+00:00,24.0,47.0,463.0,False,2021-10-06 12:00:26+00:00,I Didn't mean to push her off - Squid Game Roblox https://t.co/J6olOsZMj5 @youtube @YouTubeGaming @roblox\n\n#roblox… https://t.co/oE9P1UU2iE,Twitter Web App,False,YouTubeGaming


In [17]:
# We create a new pandas DataFrame ('squidTweets') with only the variables we need for our analysis. 
squidTweets = squidDF[['name','mentions']]

In [18]:
# We convert our pandas DataFrame ('squidTweets') into a spark DataFrame ('SquidGameTweets'), so we are now able to process this information with GraphFrames.
SquidGameTweets=spark.createDataFrame(squidTweets)
SquidGameTweets.printSchema()
SquidGameTweets.show()

root
 |-- name: string (nullable = true)
 |-- mentions: string (nullable = true)

+--------------------+---------------+
|                name|       mentions|
+--------------------+---------------+
|       Peyman 🅚🅐🅘|   KardiaChain…|
|              Yemzxy|  honourworldng|
|           EarlyGame|           Xbox|
|             Charlie|        youtube|
|             Charlie|  YouTubeGaming|
|             Charlie|         roblox|
|              Finder|      netflix's|
|       Malcolm Scott|   Kathryn_Fisk|
|🌊🌊Becca Ashley ...|       hodakotb|
|             제로 ◡̈| LouisVuitton’s|
|        DogFarm Coin|    DogFarmCoin|
|        Nikki Jacobs|      NetflixSA|
|      Suterusu Daioh| VAMichaelaLaws|
|      Monbebe Nation|OfficialMonstaX|
|           Ruben Rbg|  thisisinsider|
|            Gitartsy|         UNILAD|
|            Gitartsy|       ladbible|
|        SarahGen | ☕| millionaireeth|
|            Gitartsy|         UNILAD|
|   Valtteri Aaltonen|alessiadelvasto|
+--------------------+------

In [19]:
# We obtain an insight on how many valid tweets we have left for effective analysis after our data cleaning .
SquidGameTweets.count()

12560

<a id='2.4'></a>
### 2.4 Create the nodes & edges DataFrame

This step is important since it will allow us to crete the variables 'vertices' and 'edges' which are indispensable inputs needed for GraphFrame to be able to process and extact meaningful insights and identifying relationships in the data.

In [20]:
# We create the vertices/nodes.
tweets_vertices = (SquidGameTweets.select("name").distinct().withColumnRenamed("name","id"))
print(tweets_vertices.count())

# We create the edges.
tweets_edges = (SquidGameTweets.withColumnRenamed("name","src").withColumnRenamed("mentions","dst").groupBy("src","dst").agg(count("*").alias("occurrences")))
print(tweets_edges.count())

6072


10045


<a id='2.5'></a>
### 2.5 Create the GraphFrame

In [21]:
# We import GraphFrame
from graphframes import GraphFrame

# We create the graphframe framework needed for answering our questions about the SquidGame conversation.
tweets_graph = GraphFrame(tweets_vertices, tweets_edges)
tweets_graph.cache()

GraphFrame(v:[id: string], e:[src: string, dst: string ... 1 more field])

<a id='3'></a>
# 3. Analytics

In this notebook, we will be answering 6 essential business questions for both the marketing and sales team to better understand the conversation of SquidGame on Twitter.

<a id='3.1'></a>
**1. Conversation: Which are the top 10 most common account pairs in the SquidGame conversation?**

In [22]:
tweets_graph.edges.orderBy(desc("occurrences")).limit(10).toPandas()

,src,dst,occurrences
0,marziolowe,opensea,82
1,Sale X Low,katyperry,61
2,Sale X Low,cristiano,49
3,Sale X Low,rihanna,47
4,Sale X Low,ArianaGrande,47
5,Sale X Low,barackobama,45
6,MLN PXL,withFND!,41
7,Sale X Low,TheEllenShow,40
8,Sale X Low,taylorswift13,37
9,Sale X Low,justinbieber,35


<a id='3.2'></a>
**2. Conversation: Which are the top 10 accounts with more triangles in the SquidGame conversation?**

In [23]:
tweets_graph.triangleCount().orderBy(desc("count")).limit(10).toPandas()

,count,id
0,0,#EnjoyDigitAll
1,0,(((#RealJewNews+)))💉☠️
2,0,@itinerant
3,0,Alt-TL Hank
4,0,Bayzoo Bawra
5,0,Best Vision TV
6,0,Bhadmus Teniola Mubarak
7,0,CrossingAfterDark
8,0,Deep State Gypsy Skittle Party
9,0,Dubai Tourism


<a id='3.3'></a>
**3. Conversation: How many communities/clusters are in the SquidGame conversation?**

In [24]:
(tweets_graph.stronglyConnectedComponents(maxIter=10)
     .sort("component")
     .groupby("component")
     .agg(collect_list("id").alias("accounts"))).toPandas()

,component,accounts
0,0,[AdaletPlatformu #TürkiyeAileMeclisi @iYiLiK.org.TR]
1,1,[Adam Williams]
2,2,[Arfath Dar]
3,3,[Ayeshay]
4,4,[Belinda Zhou]
5,5,[Chandler Poling is ready for Halloween]
6,6,[CryptoFamousOrign]
7,7,[Deanna Kotrla]
8,8,"[Fin, the Other Fangbearer]"
9,9,[Gary Buglass]


<a id='3.4'></a>
**4. Conversation: Which are the top 5 most important accounts in the SquidGame conversation?**

In [25]:
tweets_rank = tweets_graph.pageRank(resetProbability=0.15, maxIter=10)

In [26]:
tweets_rank.vertices.orderBy(desc("pagerank")).limit(5).toPandas()

,id,pagerank
0,SquidGodFinance,23.628059
1,ACSupremacy,6.577052
2,toysrevil,6.577052
3,cainecapri,6.577052
4,laflarenyinc,6.577052


In [27]:
tweets_rank.edges.limit(5).toPandas()

,src,dst,occurrences,weight
0,Nadim Hossain,SquidGodFinance,1,1.0
1,Ahli Syurga,SquidGodFinance,1,1.0
2,MD MOMINUL ISLAM🎏,SquidGodFinance,1,1.0
3,John Doge,PunkPikazzo,1,1.0
4,❤⃝sᷤhͪaⷶkᷜiͥlᷝ ❤⃝⃝aⷶhͪmͫeͤdͩ.near 🎣🎏 #KOII,SquidGodFinance,1,1.0


<a id='3.5'></a>
**5. Conversation: Which are the top 5 accounts that mentioned other accounts the most in the SquidGame conversation?**

In [28]:
tweets_graph.outDegrees.orderBy(col("outDegree").desc()).limit(5).toPandas()

,id,outDegree
0,John Doge,207
1,michael jackson,68
2,marziolowe,62
3,Crypto Mushroomz,55
4,WINK_PANDA,50


<a id='3.6'></a>
**6. Conversation: Which are the top 5 accounts that were mentioned the most in the SquidGame conversation?**

In [29]:
tweets_graph.inDegrees.orderBy(col("inDegree").desc()).limit(5).toPandas()

,id,inDegree
0,netflix,872
1,GoGoSquidGame,196
2,YouTube,122
3,NetflixIndia,93
4,SunflameIndia,83


<a id='4'></a>
# 4. Export graph data
<p>If we need to export the data for graphing purposes  we can use the datasource API to create the files. These files can be used with tools like <a href='https://gephi.org/'>Gephi</a> to create graph visualizations.

In [30]:
tweets_vertices.coalesce(1).write.mode("overwrite").option("header","true").csv("hdfs://localhost:9000/export/SquidGame_tweets_vertices/")
tweets_edges.coalesce(1).withColumnRenamed("src","Source")\
                   .withColumnRenamed("dst","Destination")\
                   .write.mode("overwrite").option("header","true").csv("hdfs://localhost:9000/export/SquidGame_tweets_edges/")

<a id='5'></a>
# 5. Please, terminate the tools used
<p>Once we have completed this notebook we can stop all the services.

<a id='5.1'></a>
### 5.1 Stop Hadoop application
<ul>
    <li><p><b>Stops the Hadoop service by opening a terminal and execute:</b>:
        <br/><em>\$ hadoop-stop.sh</p></em><br/></li>